In [1]:
import sqlite3
import csv
import os
import random
import timeit

In [2]:
# 各種設定
# Configurations
DBFILE = 'TESTINDEX.db'
CSVFILE = 'refsnp_chrY.csv'
N = 1000

In [3]:
# CSVファイルからデータベースファイルを作成
# Create database file from CSV
if os.path.exists(DBFILE):
    os.unlink(DBFILE)

db = sqlite3.connect(DBFILE)
c = db.cursor()

c.execute("""
          CREATE TABLE mutations(
            id INTEGER PRIMARY KEY, 
            pos INTEGER NOT NULL,
            base_from TEXT NOT NULL,
            base_to TEXT NOT NULL);
          """)

c.execute("BEGIN;")
with open(CSVFILE) as f:
    reader = csv.reader(f)
    next(reader) # skip first line
    next(reader) # skip second line
    for row in reader:
        c.execute("INSERT INTO mutations(id, pos, base_from, base_to) VALUES(?, ?, ?, ?);", (int(row[0]), int(row[1]), row[2], row[3]))
c.execute("COMMIT;")
db.close()

In [7]:
# インデックスの作成 (インデックスを作成しない場合はこのセルをスキップすること)
# Create index (If you don't want to create an index, skip this cell)
db = sqlite3.connect(DBFILE)
c = db.cursor()

# c.execute("pos列へのインデックスを追加するSQL文をここに書く/Write an SQL statement that adds index for pos column;")
c.execute("CREATE INDEX pos_index ON mutations(pos)")

db.close()

OperationalError: index pos_index already exists

In [5]:
# SELECT文による条件抽出に要する時間を計測
# Measure the time performance of conditional query using SELECT statement
db = sqlite3.connect(DBFILE)
c = db.cursor()

# start = timeit.default_timer()
minv, maxv = c.execute("SELECT MIN(pos), MAX(pos) FROM mutations;").fetchall()[0]
# end = timeit.default_timer()
# print("Max/min finding time: ", (end - start) * 1000)

start = timeit.default_timer()
for _ in range(N):
    pos = random.randint(minv, maxv)
    c.execute("SELECT * FROM mutations WHERE pos BETWEEN ? AND ?;", (pos, pos+100))
end = timeit.default_timer()

db.close()

print("Avg time elapsed for each SELECT: {} milliseconds".format((end - start) / N * 1000))

Max/min finding time:  270.69659999961004
Avg time elapsed for each SELECT: 130.63435290000007 milliseconds
